In [4]:
import glob, os, gc
import os.path
import csv
import numpy as np
from time import time
from subprocess import (call, Popen, PIPE)
from itertools import product
from IPython.display import Image
from PIL import Image
from IPython.display import Image as IPImage
import shutil
import re
import xml.etree.ElementTree as ET
import time



##Path to Data
basepath = "/home/ubuntu/efs/SLAV_Data/" 
narrowpeak = "-ready_peaks.narrowPeak"
peaks_merged = "_peaksMerged.txt"
peaks_merged_bed = "_peaksMerged.bed"
peaks_correct_bed = "_peaksCorrect.bed"
peakregions_sml = ".peakregions_sml"
peakregions_lrg = ".peakregions_lrg"
peaks_correct_data = "_peaksCorrect.data"
peaks_L1HS_bedgraph = "_peaks_L1HS_mapped.bedgraph"
loci_sml = ".loci_sml"
loci_lrg = ".loci_lrg"
overlap = "_overlap_"
overlap_sml = "_overlap_sml_"
overlap_lrg = "_overlap_lrg_"
L1HS_bam = "-L1HS_mapped.bam"
bam = "-ready.bam"
igv = "-igv.xml"
bed = ".bed"
## rmask Paths 
L1HS = "/home/ubuntu/efs/SLAV_Data/rmask_L1HS_Final.bed"
L1PA2345 = "/home/ubuntu/efs/SLAV_Data/rmask_L1PA2345_Final.bed"
L1_Other = "/home/ubuntu/efs/SLAV_Data/rmask_L1_Other_Final.bed"
##IGV Template
IGV = "/home/ubuntu/efs/SLAV_Data/igv-template4.xml"

Bulk_1571_Cerebellum = "1571_cereb_BT_40_L3"
Bulk_1571_Hippocampus = "1571_hippo_BT_41_L3"
Bulk_1846_Cerebellum = "1846_cereb_BT_13_L3"
Bulk_1846_Cortex = "1846_cortex_BT_71_L3"
Bulk_1846_Hippocampus = "1846_hippo_BT_19_L3"
Bulk_1846_Liver = "1846_liver_BT_22_L3"
Bulk_5125_Cortex = "5125_cortex_BT_122_L3"
Bulk_5125_Hippocampus = "5125_hippo_BT_139_L3"
Bulk_5125_Liver = "5125_liver_BT_164_L3"

SC_1571_Hippo_All = ["1571_hippo_SC_43_L3","1571_hippo_SC_45_L3","1571_hippo_SC_46_L3","1571_hippo_SC_47_L3","1571_hippo_SC_48_L3","1571_hippo_SC_50_L3","1571_hippo_SC_51_L3","1571_hippo_SC_52_L3","1571_hippo_SC_53_L3","1571_hippo_SC_55_L3","1571_hippo_SC_56_L3","1571_hippo_SC_57_L3","1571_hippo_SC_58_L3","1571_hippo_SC_59_L3","1571_hippo_SC_61_L3","1571_hippo_SC_62_L3","1571_hippo_SC_63_L3","1571_hippo_SC_64_L3"]
SC_1846_Cortex_All = ["1846_cortex_SC_72_L3","1846_cortex_SC_73_L3","1846_cortex_SC_74_L3","1846_cortex_SC_75_L3","1846_cortex_SC_78_L3","1846_cortex_SC_79_L3","1846_cortex_SC_80_L3","1846_cortex_SC_81_L3","1846_cortex_SC_82_L3","1846_cortex_SC_83_L3","1846_cortex_SC_84_L3","1846_cortex_SC_85_L3","1846_cortex_SC_86_L3"]
SC_1846_Hippo_All = ["1846_hippo_SC_100_L3","1846_hippo_SC_101_L3","1846_hippo_SC_102_L3","1846_hippo_SC_103_L3","1846_hippo_SC_104_L3","1846_hippo_SC_105_L3","1846_hippo_SC_106_L3","1846_hippo_SC_107_L3","1846_hippo_SC_108_L3","1846_hippo_SC_109_L3","1846_hippo_SC_110_L3","1846_hippo_SC_111_L3","1846_hippo_SC_112_L3","1846_hippo_SC_113_L3","1846_hippo_SC_88_L3","1846_hippo_SC_89_L3","1846_hippo_SC_90_L3","1846_hippo_SC_91_L3","1846_hippo_SC_92_L3","1846_hippo_SC_93_L3","1846_hippo_SC_94_L3","1846_hippo_SC_95_L3","1846_hippo_SC_99_L3"]
SC_5125_Cortex_All = ["5125_cortex_SC_125_L3","5125_cortex_SC_126_L3","5125_cortex_SC_127_L3","5125_cortex_SC_128_L3","5125_cortex_SC_129_L3","5125_cortex_SC_130_L3","5125_cortex_SC_131_L3","5125_cortex_SC_132_L3","5125_cortex_SC_133_L3","5125_cortex_SC_134_L3","5125_cortex_SC_135_L3","5125_cortex_SC_136_L3","5125_cortex_SC_138_L3"]
SC_5125_Hippo_All = ["5125_hippo_SC_140_L3","5125_hippo_SC_141_L3","5125_hippo_SC_142_L3","5125_hippo_SC_143_L3","5125_hippo_SC_144_L3","5125_hippo_SC_145_L3","5125_hippo_SC_147_L3","5125_hippo_SC_149_L3","5125_hippo_SC_150_L3","5125_hippo_SC_151_L3","5125_hippo_SC_152_L3","5125_hippo_SC_153_L3","5125_hippo_SC_154_L3","5125_hippo_SC_155_L3","5125_hippo_SC_156_L3","5125_hippo_SC_157_L3","5125_hippo_SC_158_L3","5125_hippo_SC_159_L3","5125_hippo_SC_160_L3","5125_hippo_SC_161_L3","5125_hippo_SC_162_L3","5125_hippo_SC_163_L3"]

SC_1571_Hippo_Train = ["1571_hippo_SC_45_L3","1571_hippo_SC_48_L3","1571_hippo_SC_50_L3","1571_hippo_SC_51_L3","1571_hippo_SC_52_L3","1571_hippo_SC_55_L3","1571_hippo_SC_56_L3","1571_hippo_SC_57_L3","1571_hippo_SC_58_L3","1571_hippo_SC_59_L3"]
SC_1846_Cortex_Train = ["1846_cortex_SC_73_L3","1846_cortex_SC_78_L3","1846_cortex_SC_79_L3","1846_cortex_SC_82_L3","1846_cortex_SC_83_L3","1846_cortex_SC_85_L3"]
SC_1846_Hippo_Train = ["1846_hippo_SC_102_L3","1846_hippo_SC_103_L3","1846_hippo_SC_108_L3","1846_hippo_SC_112_L3"]
SC_5125_Cortex_Validation = ["5125_cortex_SC_129_L3","5125_cortex_SC_131_L3"]
SC_5125_Hippo_Validation = ["5125_hippo_SC_141_L3","5125_hippo_SC_144_L3","5125_hippo_SC_145_L3","5125_hippo_SC_150_L3","5125_hippo_SC_159_L3","5125_hippo_SC_160_L3","5125_hippo_SC_162_L3"]


Data_Sets_Train = []
Data_Sets_Train.append([SC_1571_Hippo_Train,Bulk_1571_Hippocampus,Bulk_1571_Cerebellum])
Data_Sets_Train.append([SC_1846_Cortex_Train,Bulk_1846_Cortex,Bulk_1846_Liver])
Data_Sets_Train.append([SC_1846_Hippo_Train,Bulk_1846_Hippocampus,Bulk_1846_Liver])

Data_Sets_Validation = []
Data_Sets_Validation.append([SC_5125_Cortex_Validation,Bulk_5125_Cortex,Bulk_5125_Liver])
Data_Sets_Validation.append([SC_5125_Hippo_Validation,Bulk_5125_Hippocampus,Bulk_5125_Liver])

Data_Sets_Test = []
SC_1571_Hippo_Test = list(set(SC_1571_Hippo_All) - set(SC_1571_Hippo_Train))
SC_1846_Cortex_Test = list(set(SC_1846_Cortex_All) - set(SC_1846_Cortex_Train))
SC_1846_Hippo_Test = list(set(SC_1846_Hippo_All) - set(SC_1846_Hippo_Train))
SC_5125_Cortex_Test = list(set(SC_5125_Cortex_All) - set(SC_5125_Cortex_Validation))
SC_5125_Hippo_Test = list(set(SC_5125_Hippo_All) - set(SC_5125_Hippo_Validation))

In [ ]:
for dset in Data_Sets_Train:
    for cell in dset[0]:
        print cell
        os.chdir(os.path.join(basepath, cell))
        for file in glob.glob("*_lrg_s*__*.png"):
            newfile = re.sub("_s\d+__", "-", file)
            shutil.move(file, newfile)
            img = Image.open(newfile)
            width = img.size[0]
            height = img.size[1]
            img2 = img.crop((160,130,width,height))
            path = os.path.splitext(newfile)[0]
            basename = os.path.basename(path)
            outfile1 = basename + "_cropped.png"
            img2.save(outfile1)

1571_hippo_SC_45_L3
1571_hippo_SC_48_L3
1571_hippo_SC_50_L3
1571_hippo_SC_51_L3
1571_hippo_SC_52_L3


In [3]:
for dset in Data_Sets_Train:
    for cell in dset[0]:
        print cell
        with open(os.path.join(basepath, cell, cell+"_Input_metadata.txt")) as f:
            for line in csv.reader(f, delimiter="\t"):
                if os.path.isfile(os.path.join(basepath, cell,line[3])):
                    #print line[3]
                    filename = line[3]
                    readclass = line[5].split(":")[0]
                    peakclass_peak = line[5].split(":")[1]
                    L1_class_peak = line[5].split(":")[2]
                    peakclass_sml = line[5].split(":")[3]
                    L1_class_sml = line[5].split(":")[4]
                    class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml]
                    dst = "".join(class_sml)
                    if not os.path.exists(os.path.join(basepath, "Train", dst)):
                        os.makedirs(os.path.join(basepath, "Train", dst))
                    if not os.path.isfile(os.path.join(basepath, "Train", dst,filename)):
                        shutil.move(os.path.join(basepath, cell,filename), os.path.join(basepath, "Train", dst))        

1846_cortex_SC_73_L3
1846_cortex_SC_78_L3
1846_cortex_SC_79_L3


KeyboardInterrupt: 

In [ ]:
for dset in Data_Sets_Validation:
    for cell in dset[0]:
        print cell
        with open(os.path.join(basepath, cell, cell+"_Input_metadata.txt")) as f:
            for line in csv.reader(f, delimiter="\t"):
                if os.path.isfile(os.path.join(basepath, cell,line[3])):
                    #print line[3]
                    filename = line[3]
                    readclass = line[5].split(":")[0]
                    peakclass_peak = line[5].split(":")[1]
                    L1_class_peak = line[5].split(":")[2]
                    peakclass_sml = line[5].split(":")[3]
                    L1_class_sml = line[5].split(":")[4]
                    class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml]
                    dst = "".join(class_sml)
                    if not os.path.exists(os.path.join(basepath, "Validation", dst)):
                        os.makedirs(os.path.join(basepath, "Validation", dst))
                    if not os.path.isfile(os.path.join(basepath, "Validation", dst,filename)):
                        shutil.move(os.path.join(basepath, cell,filename), os.path.join(basepath, "Validation", dst))       

In [ ]:
for dset in Data_Sets_Test:
    for cell in dset[0]:
        print cell
        with open(os.path.join(basepath, cell, cell+"_Input_metadata.txt")) as f:
            for line in csv.reader(f, delimiter="\t"):
                if os.path.isfile(os.path.join(basepath, cell,line[3])):
                    #print line[3]
                    filename = line[3]
                    readclass = line[5].split(":")[0]
                    peakclass_peak = line[5].split(":")[1]
                    L1_class_peak = line[5].split(":")[2]
                    peakclass_sml = line[5].split(":")[3]
                    L1_class_sml = line[5].split(":")[4]
                    class_sml = [readclass,peakclass_peak,L1_class_peak,peakclass_sml,L1_class_sml]
                    dst = "".join(class_sml)
                    if not os.path.exists(os.path.join(basepath, "Test", dst)):
                        os.makedirs(os.path.join(basepath, "Test", dst))
                    if not os.path.isfile(os.path.join(basepath, "Test", dst,filename)):
                        shutil.move(os.path.join(basepath, cell,filename), os.path.join(basepath, "Test", dst))

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = os.path.join(basepath, 'Train')
validation_data_dir = os.path.join(basepath, 'Validation')
nb_train_samples = 350000
nb_validation_samples = 10000
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1734, activation='softmax'))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1./255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size, verbose=1)

model.save_weights('first_try.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
activation_16 (Activation)   (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 125, 125, 32)      9248      
_________________________________________________________________
activation_17 (Activation)   (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 60, 60, 64)        18496     
__________

In [ ]:
####Test Basic Convolution Model

import random
np.random.seed(123)  # for reproducibility
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

config = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32, allow_soft_placement=True, device_count = {'CPU': 32})
session = tf.Session(config=config)
K.set_session(session)

# 6. Preprocess class labels
Ydict_1 = {}
Ydict_2 = {}
Y_unique = np.unique(Y_sml)
print Y_unique
i=0
for y_u in Y_unique:
    Ydict_1[i] = y_u
    Ydict_2[y_u] = i
    i+=1
print Ydict_2
trainsize = 20000
testsize = 10000
train_test = random.sample(range(0, n_samples_sml),trainsize+testsize)
train = train_test[0:trainsize]
test = train_test[trainsize:trainsize+testsize]
X_train = []
X_test = []
X_train = X_sml[train,:]
X_test = X_sml[test,:]
Ytr = [Ydict_2[y_train] for y_train in Y_sml[train]]
Yte = [Ydict_2[y_test] for y_test in Y_sml[test]]
Y_train = np_utils.to_categorical(Ytr)
Y_test = np_utils.to_categorical(Yte)

num_categories = len(np.unique(Y_sml))
print num_categories
# 7. Define model architecture
model = Sequential() 
model.add(Conv2D(32, (3,3), activation='relu', data_format="channels_first", input_shape=(3,500,200)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_categories, activation='softmax'))

# 8. Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

X_train = X_train.reshape(X_train.shape[0], 3, 500, 200)
X_test = X_test.reshape(X_test.shape[0], 3, 500, 200)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 9. Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

score = model.evaluate(X_test, Y_test, verbose=1)

score

classes = model.predict(X_test, batch_size=128)

classes

X_test.shape

classes.shape

classes[1,]

In [ ]:
print score

In [ ]:
print classes.shape